# Example Visualizations

This notebook provides example queries and visualizations.

## Setup
The cell below is used to 
* import required libraries
* setting up the connection to the Neo4j database
* define the D3 based HTML template for custom visualizations

In [ ]:
import pandas as pd 
import plotly.express as px
import pygal as pg
from pathlib import Path
from string import Template
from IPython.display import HTML, Javascript, display

neo4j_url=%env NEO4J_URL
from py2neo import Graph
graph = Graph(neo4j_url + '/db/data')

In [ ]:
d3v4 = Path('../vis/lib/d3.v4.min.js').read_text()
d3v4 = "<script type='text/javascript'>" + d3v4 + "</script>"
display(HTML(d3v4))

In [ ]:
lodash = Path('../vis/lib/lodash.min.js').read_text()
lodash = "<script type='text/javascript'>" + lodash + "</script>"
display(HTML(lodash))

In [ ]:
display(HTML(filename='../vis/chord/chord.css.html'))

In [ ]:
display(HTML(filename='../vis/circle-packing/circle-packing.css.html'))

## Table
The simplest visualization is a table, the rows and columns are rendered directly from the result returned by the query.

In [ ]:
graph.run('''
    MATCH  (a:Artifact)-[:CONTAINS]->(n:Type) 
    RETURN a.fqn as Artifact, 
           count(n) as TypesPerArtifact
''').to_table()    

## Pie Chart
A pie chart is used for illustrating proportions, e.g. artifact sizes. Therefore the query returns a row per attifact, each containing the name and the number of contained types.

In [ ]:
artifactSizes = graph.run('''
    MATCH (artifact:Artifact)-[:CONTAINS]->(type:Type)
    RETURN coalesce(artifact.fqn, artifact.fileName) as Artifact, 
           count(type) AS Types
''').to_data_frame()           

px.pie(artifactSizes, values='Types', names='Artifact', title='Artifact Size')

## Bar Chart and Stacked Bar Chart
Bar charts are another way to visualize proportions. The example query below returns an artifact per row, each containg each containing the name of the artifact and the number of contained types.

In [ ]:
px.bar(artifactSizes, x='Artifact', y='Types', title='Artifact Size')  

Bar charts may be stacked, e.g. to visualize the different Java class types (i.e. class, interface, enum or annotation) per artifact. The query therefore is extended by a column `JavaType` which determines the color. 

In [ ]:
artifactSizesByType = graph.run('''
    MATCH  (artifact:Artifact)-[:CONTAINS]->(type:Type)
    RETURN coalesce(artifact.fqn, artifact.fileName) as Artifact, 
           case
               when type:Class then 'Class' 
               when type:Interface then 'Interface' 
               when type:Enum then 'Enum' 
               when type:Annotation then 'Annotation' 
               end as JavaType, count(type) as Types 
    ORDER BY Types desc
''').to_data_frame()    

px.bar(artifactSizesByType, x='Artifact', color='JavaType', y='Types', title='Artifact Size')

## Circle Packing
A circle packing diagram can be used to illustrate hierarchical structures, e.g. packages and their children. The query returns a flattened tree structure containing one row per parent/child-combination with four columns:
* *Parent_Fqn*: the fully qualified name of the parent (e.g. type name including package name)
* *Parent_Name*: the name of the parent (e.g. type name without package name)
* *Child_Fqn*: the fully qualified name of the child
* *Child_Is_Leaf*: a boolean value that if true indicates that the child has no further children (e.g. true for a type, false for a package)

In [ ]:
packageHierarchy = graph.run('''
    MATCH  (package:Package)-[:CONTAINS]->(element)
    WHERE  (package)-[:CONTAINS*]->(:Type) and exists(element.fqn) AND (element:Type OR (element)-[:CONTAINS*]->(:Type))
    WITH   package, element, element:Type as leaf
    RETURN DISTINCT package.fqn AS Parent_Fqn, 
           package.name AS Parent_Name, 
           element.fqn AS Child_Fqn, 
           element.name AS Child_Name, 
           leaf AS Child_Is_Leaf
''').to_data_frame()     

packageHierarchyCsv = '\"' + packageHierarchy.to_csv(index = False).replace("\r\n","\n").replace("\n","\\n") + '\"'

In [ ]:
circlePackingScript = Template(Path('../vis/circle-packing/circle-packing.js').read_text())

Javascript(circlePackingScript.substitute(data=packageHierarchyCsv))

## Treemap

A treemap is another way of visualizing hierarchical structures. Each element is represented by a rectangle, the size and the color represent metrics per element. The example query returns a flattened tree containing one row per package:

* *Element*: The name of the element to be displayed as rectangle
* *Parent*: The name of the element's parent (optional for root elements)
* *Size*: Determines the relative size of the rectangle
* *Color*: Determines the color of the rectangle

In [ ]:
packageTree = graph.run('''
    MATCH (package:Package)
    OPTIONAL MATCH (parent:Package)-[:CONTAINS]->(package)
    OPTIONAL MATCH (package)-[:CONTAINS]->(type:Type)
    OPTIONAL MATCH (type)-[:DECLARES]->(method:Method)
    RETURN package.fqn as Element, parent.fqn as Parent, count(type) as Size, sum(method.effectiveLineCount) as Color
''').to_data_frame()

px.treemap(packageTree, names = 'Element', parents = 'Parent', values = 'Size', color= 'Color')

## Chord Diagram
A chord diagram is used to illustrate dependencies between elements, e.g. packages. The query for each dependency returns
* *Source*: The name of the dependent element (e.g. source package)
* *Target*: The name of the element's dependency (e.g. target package)
* *X_Count*: The weight of of the dependency (e.g. the coupling between both packages)

In [ ]:
packageDependencies = graph.run('''
    MATCH  (p1:Package)-[:CONTAINS]->(t1:Type),
           (p2:Package)-[:CONTAINS]->(t2:Type),
           (t1)-[dep:DEPENDS_ON]->(t2)
    WHERE  p1 <> p2
    RETURN p1.name AS Source,
           p2.name AS Target,
           COUNT(dep) AS X_Count
''').to_data_frame()           

packageDependenciesCsv = '\"' + packageDependencies.to_csv(index = False).replace("\r\n","\n").replace("\n","\\n") + '\"'

In [ ]:
chordScript = Template(Path('../vis/chord/chord.js').read_text())

Javascript(chordScript.substitute(data=packageDependenciesCsv))